In [10]:
import keras as keras
import tensorflow as tf
from keras import layers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from keras.layers import Bidirectional
from keras.utils import pad_sequences
from keras.callbacks import EarlyStopping
from keras.callbacks import ReduceLROnPlateau
from keras import regularizers
from keras.layers import Input, LSTM, Dense, Concatenate
from keras.models import Model
from itertools import permutations
import keras.backend as K
import pandas as pd
import numpy as np

In [3]:
def permutation_invariant_loss(y_true, y_pred):
    # Sort the true and predicted values along the last axis
    y_true_sorted = tf.sort(y_true, axis=-1)
    y_pred_sorted = tf.sort(y_pred, axis=-1)

    # Calculate the mean absolute error between the sorted true and predicted values
    return K.mean(K.abs(y_true_sorted - y_pred_sorted), axis=-1)
class LRHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.lr = []

    def on_epoch_end(self, batch, logs={}):
        optimizer = self.model.optimizer
        lr = keras.backend.get_value(optimizer.lr)
        self.lr.append(lr)

In [4]:
csv_file_path = 'sinusoid_dataset.csv'
df = pd.read_csv(csv_file_path)
features = df['Feature'].apply(lambda x: np.array([float(val.strip("[]")) for val in x.split()])).values
labels = df['Label'].apply(lambda x: np.array([float(val.strip("[]")) for val in x.split(',')])).values
print(features.shape)
print(labels.shape)

(200000,)
(200000,)


In [5]:
padded_array = pad_sequences(labels, padding='post', maxlen=3)
print(padded_array.shape)
labels = np.vstack(padded_array)
features = np.vstack(features)
#scaler = MinMaxScaler()
#features = scaler.fit_transform(features)

(200000, 3)


In [6]:
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

X_train_tensor = tf.convert_to_tensor(X_train, dtype=tf.float32)
y_train_tensor = tf.convert_to_tensor(y_train, dtype=tf.float32)
X_test_tensor = tf.convert_to_tensor(X_test, dtype=tf.float32)
y_test_tensor = tf.convert_to_tensor(y_test, dtype=tf.float32)

print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train_categorical shape:", y_train.shape)
print("y_test_categorical shape:", y_test.shape)

2023-12-10 16:44:34.317689: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:08:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-12-10 16:44:34.351277: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:08:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-12-10 16:44:34.351344: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:08:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-12-10 16:44:34.354800: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:08:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-12-10 16:44:34.354891: I external/local_xla/xla/stream_executor

X_train shape: (160000, 299)
X_test shape: (40000, 299)
y_train_categorical shape: (160000, 3)
y_test_categorical shape: (40000, 3)


2023-12-10 16:44:34.592954: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 191360000 exceeds 10% of free system memory.


In [7]:
# Define three LSTM branches
inputs = keras.Input(shape=(299,1))
# Branch 1
lstm_branch_1 = LSTM(units=64, return_sequences=True)(inputs)
flatten_1 = layers.Flatten()(lstm_branch_1)
output_branch_1 = Dense(units=1, activation='linear', name='output_branch_1')(flatten_1)

# Branch 2
lstm_branch_2 = LSTM(units=64, return_sequences=True)(lstm_branch_1)  # Pass the output of branch 1
flatten_2 = layers.Flatten()(lstm_branch_2)
output_branch_2 = Dense(units=1, activation='linear', name='output_branch_2')(flatten_2)

# Branch 3
lstm_branch_3 = LSTM(units=64, return_sequences=True)(lstm_branch_2)  # Pass the output of branch 2
flatten_3 = layers.Flatten()(lstm_branch_3)
output_branch_3 = Dense(units=1, activation='linear', name='output_branch_3')(flatten_3)

# Concatenate the outputs of the three branches
merged_output = Concatenate(axis=-1)([output_branch_1, output_branch_2, output_branch_3])

# Define the model
model = Model(inputs=inputs, outputs=merged_output)
print(model.summary())

2023-12-10 16:44:38.266101: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 299, 1)]             0         []                            
                                                                                                  
 lstm (LSTM)                 (None, 299, 64)              16896     ['input_1[0][0]']             
                                                                                                  
 lstm_1 (LSTM)               (None, 299, 64)              33024     ['lstm[0][0]']                
                                                                                                  
 lstm_2 (LSTM)               (None, 299, 64)              33024     ['lstm_1[0][0]']              
                                                                                              

In [8]:
model.compile(optimizer='adam',
              loss=permutation_invariant_loss)
#overfitCallback = EarlyStopping(monitor='loss', min_delta=0, patience = 4)
#lr_scheduler = ReduceLROnPlateau(monitor='loss', factor=0.1, patience=2, min_lr=1e-7)
#lr_history = LRHistory()
model.fit(X_train_tensor, y_train_tensor, epochs=40, batch_size=256, validation_split=0.2)

test_loss = model.evaluate(X_test_tensor, y_test_tensor)
print(f'Test Loss: {test_loss:.4f}')

Epoch 1/20


2023-12-10 16:44:46.830021: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 153088000 exceeds 10% of free system memory.
2023-12-10 16:44:49.719518: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904
2023-12-10 16:44:49.824636: I external/local_xla/xla/service/service.cc:168] XLA service 0x563f8facda30 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-12-10 16:44:49.824689: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 2070 SUPER, Compute Capability 7.5
2023-12-10 16:44:49.847171: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1702244689.963269  119889 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


500/500 [==============================] - 56s 104ms/step - loss: 7.4383 - val_loss: 4.2882
Epoch 2/20
500/500 [==============================] - 53s 106ms/step - loss: 3.3062 - val_loss: 2.4147
Epoch 3/20
500/500 [==============================] - 50s 100ms/step - loss: 1.8798 - val_loss: 1.4476
Epoch 4/20
500/500 [==============================] - 51s 103ms/step - loss: 2.1338 - val_loss: 1.3871
Epoch 5/20
500/500 [==============================] - 49s 98ms/step - loss: 1.3834 - val_loss: 0.9811
Epoch 6/20
500/500 [==============================] - 50s 101ms/step - loss: 2.2459 - val_loss: 1.8390
Epoch 7/20
500/500 [==============================] - 51s 103ms/step - loss: 1.1882 - val_loss: 1.0048
Epoch 8/20
500/500 [==============================] - 53s 106ms/step - loss: 1.9497 - val_loss: 2.2392
Epoch 9/20
500/500 [==============================] - 52s 105ms/step - loss: 1.2865 - val_loss: 0.9897
Epoch 10/20
500/500 [==============================] - 50s 100ms/step - loss: 0.8616 

In [9]:
csv_file_path = 'samplesreal.csv'  # Replace with the actual path to your CSV file
df = pd.read_csv(csv_file_path)

# Extract features and labels
features = df['Feature'].apply(lambda x: np.array([float(val.strip("[]")) for val in x.split(',')])).values
labels = [[label] for label in df['Label'].tolist()]
# Convert the string of up to 5 numbers to a NumPy array of floats

padded_array = pad_sequences(labels, padding='post', maxlen=3)
labels = np.vstack(padded_array)
features = np.vstack(features)
means = np.mean(features, axis=1, keepdims=True)
features = features - means
features = features * 50

print("Real Sinusoid Tests:\n")
for i in range(8):
  print(labels[i:i+1])
  print(model.predict(features[i:i+1]))

print("Synthetic Sinusoid Tests:\n")
for i in range(100):
  print(y_test[i:i+1])
  print(model.predict(X_test[i:i+1]))

Real Sinusoid Tests:

[[1 0 0]]
1/1 [==============================] - 1s 715ms/step
[[ 3.1865878 39.732574   2.5606937]]
[[10  0  0]]
1/1 [==============================] - 0s 89ms/step
[[3.430654   8.469257   0.52748436]]
[[19  0  0]]
1/1 [==============================] - 0s 35ms/step
[[16.42772    3.7267475  0.0757518]]
[[27  0  0]]
1/1 [==============================] - 0s 33ms/step
[[21.412746   14.580589    0.40184122]]
[[36  0  0]]
1/1 [==============================] - 0s 67ms/step
[[24.148848  58.108757  -0.6507648]]
[[45  0  0]]
1/1 [==============================] - 0s 48ms/step
[[31.495916  86.858765  -0.6639408]]
[[54  0  0]]
1/1 [==============================] - 0s 36ms/step
[[ 39.618366   104.07333     -0.61309683]]
[[90  0  0]]
1/1 [==============================] - 0s 43ms/step
[[ 87.705894  131.08492    -0.8982781]]
Synthetic Sinusoid Tests:

[[69  0  0]]
1/1 [==============================] - 0s 64ms/step
[[69.2595     -0.16325805  0.07835612]]
[[60 75 31]]
1/1 [==